In [ ]:
import pydicom
from fastai.vision.all import *
np.random.seed(20210717)

In [ ]:
def read_data(p):
    dicom = pydicom.read_file(p)
    return dicom.pixel_array.astype(np.float32)

def load_volumes(id, modes=['T2w', 'T1wCE', 'T1w', 'FLAIR']):
    p = Path(f'/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{id:05d}')
    slices = {m:np.sort([int(o.stem.split('-')[1]) for o in (p/m).iterdir()]) for m in modes}
    volumes = {}
    for m in modes:
        ims = []
        for s in slices[m]:
            ims.append(read_data(p/m/f'Image-{s}.dcm'))
        volumes[m] = np.array(ims)
    return volumes

def plot_volumes(volumes):
    fig, axes = plt.subplots(ncols=10, nrows=4, figsize=(15,9))
    for i, (m, v) in enumerate(volumes.items()):
        axes[i,5].set_title(m)
        for j in range(10):
            axes[i,j].imshow(v[np.linspace(0,len(v)-1,10).astype(np.uint32)[j]])
            axes[i,j].axis('off')
    fig.tight_layout(pad=0)
    plt.show()

In [ ]:
path = Path('/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification')
path_train = path/'train'
path_test = path/'test'

train_df = pd.read_csv(path/'train_labels.csv')
sub_df = pd.read_csv(path/'sample_submission.csv')

In [ ]:
for i in progress_bar(np.random.choice(range(len(train_df)), 50)):
    print(f'Plotting sample {i}')
    ref = train_df.loc[i, 'BraTS21ID']
    volumes = load_volumes(ref)
    plot_volumes(volumes)